## **Federated Learning for attack detection: 5 nodes sharing gradients: KRUM**

IDs from this file = **idKRUMxy** (x = 0 if experiment with dataset, x = 1 if epochs & iterations, y being integer equal or greater than 0)

In this file, experiments with different datasets, and number of epochs & iterations are done. The experiments are divided into sections, based on the elements being changed. 

### Static elements for all experiments (execute first)

In [1]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Extracting labels 
    data_labels = data[['label']]

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)

    # Create a complementary field of attack & Transform to One hot encoding - LABELS
    normal=data_labels['label']
    normal=normal.replace(1,2)
    normal=normal.replace(0,1)
    normal=normal.replace(2,0)

    # Insert the new column in data labels
    data_labels.insert(1, 'normal', normal)
    data_labels = pd.get_dummies(data_labels)

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Value given for the missing columns
    auxCol=0

    # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [3]:
# Model building and definition
def build_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=32,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 2), padding='same'))
    model.add(layers.Conv2D(filters=64,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 2), padding='same'))
    model.add(layers.Flatten())
    model.add(Dense(444, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    return model 

In [4]:
# Returns values of loss, accuracy, f1, precision and recall of model evaluating with test dataset 
def evaluation(model, x, y): 
    loss, accuracy = model.evaluate(x, y)
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y = np.argmax(y, axis=1)
    report = classification_report(y, y_pred, target_names=['normal', 'attack'], output_dict=True)
    # Obtain f1, precision and recall from the report
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    return loss, accuracy, f1, precision, recall

In [5]:
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')
test_complete = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Complete.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_16076/3457440085.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_16076/3457440085.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_complete = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Complete.csv')


In [6]:
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)
xcomplete, ycomplete = preprocessing(test_complete)

### Experiments with datasets

#### Dataset Filt5A

In [7]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part1.csv', low_memory=False)
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part2.csv', low_memory=False)
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part3.csv', low_memory=False)
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part4.csv', low_memory=False)
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part5.csv', low_memory=False)

In [8]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [9]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idKRUM00.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [10]:
def aggregate(grad_list, num_mal = 0):
    
    num_to_consider = num_nodes - num_mal - 2

    # Flatten gradients to compute distances
    flat_grads = [tf.concat([tf.reshape(g, [-1]) for g in grad], axis=0).numpy() for grad in grad_list]

    # Compute pairwise squared Euclidean distances
    distances = np.zeros((num_nodes, num_nodes))
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            dist = np.sum((flat_grads[i] - flat_grads[j]) ** 2)
            distances[i, j] = dist
            distances[j, i] = dist

    # Find the Krum gradient
    krum_scores = []
    for i in range(num_nodes):
        sorted_distances = np.sort(distances[i])
        score = np.sum(sorted_distances[:num_to_consider])
        krum_scores.append(score)
    
    krum_index = np.argmin(krum_scores)
    selected_grad = grad_list[krum_index]

    return selected_grad

In [11]:
global_model = build_model((24,1,1))

In [12]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [13]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [14]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)
    loss_complete, accuracy_complete, f1_complete, precision_complete, recall_complete = evaluation(global_model, xcomplete, ycomplete)

    loss_it.append([loss_basic, loss_plus, loss_complete])
    accuracy_it.append([accuracy_basic, accuracy_plus, accuracy_complete])
    f1_it.append([f1_basic, f1_plus, f1_complete])
    precision_it.append([precision_basic, precision_plus, precision_complete])
    recall_it.append([recall_basic, recall_plus, recall_complete])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idKRUM00.hdf5')

Epoch 1/5
31/31 [==============================] - 11s 342ms/step - loss: 0.3642 - accuracy: 0.9667 - val_loss: 1.8792 - val_accuracy: 0.6196
Epoch 2/5
31/31 [==============================] - 9s 289ms/step - loss: 0.0594 - accuracy: 0.9916 - val_loss: 0.8933 - val_accuracy: 0.6202
Epoch 3/5
31/31 [==============================] - 9s 279ms/step - loss: 0.0299 - accuracy: 0.9923 - val_loss: 0.7760 - val_accuracy: 0.6249
Epoch 4/5
31/31 [==============================] - 9s 280ms/step - loss: 0.0272 - accuracy: 0.9923 - val_loss: 0.5877 - val_accuracy: 0.6559
Epoch 5/5
31/31 [==============================] - 8s 246ms/step - loss: 0.0259 - accuracy: 0.9923 - val_loss: 0.7493 - val_accuracy: 0.6244
Epoch 1/5
31/31 [==============================] - 7s 235ms/step - loss: 0.0419 - accuracy: 0.9897 - val_loss: 0.4992 - val_accuracy: 0.7400
Epoch 2/5
31/31 [==============================] - 7s 236ms/step - loss: 0.0275 - accuracy: 0.9912 - val_loss: 0.5183 - val_accuracy: 0.6632
Epoch 3/5
31

In [15]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.08878409117460251, 0.37367090582847595, 0.14826644957065582], [0.05707963928580284, 0.296873539686203, 0.09621022641658783], [0.08165192604064941, 0.6474021077156067, 0.15797719359397888]]
Accuracy for iterations:  [[0.9639633297920227, 0.8402600884437561, 0.9347139000892639], [0.9731733202934265, 0.8620940446853638, 0.9568177461624146], [0.9693023562431335, 0.7406452298164368, 0.9428328275680542]]
F1 for iterations:  [[0.9638645940345658, 0.8335965860492922, 0.9344801079863494], [0.9731358370953441, 0.857684002050284, 0.9567667355664131], [0.9692349378527148, 0.7125105552563743, 0.9426773587612735]]
Precision for iterations:  [[0.9656154245460951, 0.8680512788671989, 0.9410082055066445], [0.9737565088890403, 0.8826517217546718, 0.9589824408983635], [0.9705001595482861, 0.8138686114556182, 0.9476890554678995]]
Recall for iterations:  [[0.9639633059686231, 0.8402601177068953, 0.934713920024552], [0.9731733325542669, 0.8620940201990845, 0.9568177221017096], [0.9

#### Dataset Filt5B

In [16]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part1.csv', low_memory=False)
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part2.csv', low_memory=False)
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part3.csv', low_memory=False)
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part4.csv', low_memory=False)
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part5.csv', low_memory=False)

In [17]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [18]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idKRUM01.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [19]:
global_model = build_model((24,1,1))

In [20]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [21]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [22]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)
    loss_complete, accuracy_complete, f1_complete, precision_complete, recall_complete = evaluation(global_model, xcomplete, ycomplete)

    loss_it.append([loss_basic, loss_plus, loss_complete])
    accuracy_it.append([accuracy_basic, accuracy_plus, accuracy_complete])
    f1_it.append([f1_basic, f1_plus, f1_complete])
    precision_it.append([precision_basic, precision_plus, precision_complete])
    recall_it.append([recall_basic, recall_plus, recall_complete])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idKRUM01.hdf5')

Epoch 1/5
31/31 [==============================] - 18s 583ms/step - loss: 0.3610 - accuracy: 0.9093 - val_loss: 2.2652 - val_accuracy: 0.5512
Epoch 2/5
31/31 [==============================] - 14s 451ms/step - loss: 0.0586 - accuracy: 0.9916 - val_loss: 1.1078 - val_accuracy: 0.5527
Epoch 3/5
31/31 [==============================] - 14s 457ms/step - loss: 0.0294 - accuracy: 0.9922 - val_loss: 0.9290 - val_accuracy: 0.5569
Epoch 4/5
31/31 [==============================] - 11s 367ms/step - loss: 0.0266 - accuracy: 0.9922 - val_loss: 0.7739 - val_accuracy: 0.5710
Epoch 5/5
31/31 [==============================] - 11s 370ms/step - loss: 0.0255 - accuracy: 0.9921 - val_loss: 0.8235 - val_accuracy: 0.5605
Epoch 1/5
31/31 [==============================] - 11s 359ms/step - loss: 0.0374 - accuracy: 0.9903 - val_loss: 0.5398 - val_accuracy: 0.6966
Epoch 2/5
31/31 [==============================] - 10s 328ms/step - loss: 0.0258 - accuracy: 0.9914 - val_loss: 0.4068 - val_accuracy: 0.7266
Epoch 

In [23]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.1294107586145401, 0.22612498700618744, 0.18356190621852875], [0.09662602841854095, 0.23518449068069458, 0.1565445512533188], [0.08443371206521988, 0.1928814947605133, 0.13982537388801575]]
Accuracy for iterations:  [[0.9241140484809875, 0.8864709734916687, 0.8913781642913818], [0.9472669363021851, 0.8825655579566956, 0.9152536392211914], [0.9557246565818787, 0.9300661087036133, 0.926755428314209]]
F1 for iterations:  [[0.9233918735333838, 0.8838986349720896, 0.8901692764377828], [0.9469940166524514, 0.8797686921800936, 0.9147101698666007], [0.955552926164441, 0.9293800076359917, 0.9264159913216581]]
Precision for iterations:  [[0.9326584387033396, 0.9000277455110332, 0.9094021627378143], [0.951245544478366, 0.8969437182595985, 0.9261135945657125], [0.9584258577276722, 0.9344163489049463, 0.9347770819443417]]
Recall for iterations:  [[0.924114055333197, 0.8864709728983506, 0.8913781919382852], [0.9472669373922697, 0.8825655743660539, 0.9152536461857166], [0.955

#### Dataset Filt5C

In [24]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5C-Part1.csv', low_memory=False)
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5C-Part2.csv', low_memory=False)
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5C-Part3.csv', low_memory=False)
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5C-Part4.csv', low_memory=False)
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5C-Part5.csv', low_memory=False)

In [25]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [26]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idKRUM02.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [27]:
global_model = build_model((24,1,1))

In [28]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [29]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [30]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)
    loss_complete, accuracy_complete, f1_complete, precision_complete, recall_complete = evaluation(global_model, xcomplete, ycomplete)

    loss_it.append([loss_basic, loss_plus, loss_complete])
    accuracy_it.append([accuracy_basic, accuracy_plus, accuracy_complete])
    f1_it.append([f1_basic, f1_plus, f1_complete])
    precision_it.append([precision_basic, precision_plus, precision_complete])
    recall_it.append([recall_basic, recall_plus, recall_complete])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idKRUM02.hdf5')

Epoch 1/5
16/16 [==============================] - 4s 235ms/step - loss: 0.5476 - accuracy: 0.7475 - val_loss: 1.6460 - val_accuracy: 0.0000e+00
Epoch 2/5
16/16 [==============================] - 3s 199ms/step - loss: 0.2205 - accuracy: 0.9476 - val_loss: 3.6629 - val_accuracy: 0.1143
Epoch 3/5
16/16 [==============================] - 3s 200ms/step - loss: 0.0977 - accuracy: 0.9883 - val_loss: 4.3456 - val_accuracy: 0.1322
Epoch 4/5
16/16 [==============================] - 3s 198ms/step - loss: 0.0587 - accuracy: 0.9893 - val_loss: 2.5633 - val_accuracy: 0.1342
Epoch 5/5
16/16 [==============================] - 3s 204ms/step - loss: 0.0373 - accuracy: 0.9901 - val_loss: 1.1930 - val_accuracy: 0.2259
Epoch 1/5
14/14 [==============================] - 3s 222ms/step - loss: 0.0219 - accuracy: 0.9949 - val_loss: 0.8595 - val_accuracy: 0.7270
Epoch 2/5
14/14 [==============================] - 3s 215ms/step - loss: 0.0202 - accuracy: 0.9950 - val_loss: 0.5374 - val_accuracy: 0.7288
Epoch 3/5

In [31]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.07510484009981155, 0.15525732934474945, 0.08319155871868134], [0.06100010126829147, 0.12212061882019043, 0.06060165911912918], [0.05247027054429054, 0.09796081483364105, 0.05343760922551155]]
Accuracy for iterations:  [[0.9873133897781372, 0.9606190323829651, 0.982785701751709], [0.989467978477478, 0.9645426273345947, 0.9887214303016663], [0.990234911441803, 0.9727166891098022, 0.9891329407691956]]
F1 for iterations:  [[0.9873178303106482, 0.9606038442324906, 0.9827856759477432], [0.9894729211773826, 0.9645320799718331, 0.9887208056204078], [0.9902391931011721, 0.9726951004082693, 0.9891323871495287]]
Precision for iterations:  [[0.9874161155962561, 0.9606068016196768, 0.9827860172386974], [0.9896665365698087, 0.9645324642787586, 0.9888264897944212], [0.9904064503870083, 0.9727472144071837, 0.9892319900707868]]
Recall for iterations:  [[0.9873133892313535, 0.9606190510789798, 0.9827856789124567], [0.9894679949750211, 0.9645426142556128, 0.9887214111837287], [0